In [1]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 107.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.2 MB/s eta 0:00:00


# 텍스트 생성
## Greedy Search
![](https://miro.medium.com/v2/resize:fit:1400/1*Dqw1Y7zDcxKOBvsl_bbtog.png)

## Beam Search
![](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/beam_search.png)
![](https://www.baeldung.com/wp-content/uploads/sites/4/2021/10/beamsearch.jpg)
## Sampling
![](https://huggingface.co/blog/assets/02_how-to-generate/sampling_search.png)

### Top-k Sampling
![](https://www.researchgate.net/publication/370841754/figure/fig2/AS:11431281159484568@1684378994491/Top-K-sampling-is-insensitive-to-model-confidence.png)

### Top-p Sampling (Nucleus Sampling)
![](https://miro.medium.com/v2/resize:fit:720/format:webp/0*J37qonVPJvKZpzv2)

### Temperature Sampling
![](https://miro.medium.com/v2/resize:fit:720/format:webp/1*yJUw-zkpTfqdTVeOO7OKnQ.png)

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers

# 경고 안띄우기
transformers.logging.set_verbosity_error()

In [3]:
# 생성이 오래걸리므로 가급적 GPU로 하시길 권장드립니다
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model_id = "42dot/42dot_LLM-PLM-1.3B"
# model_id = "heegyu/ajoublue-gpt2-medium"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id).to(device).eval()
if "cuda" in device:
  model.half() # float16 으로 더 빠르게 생성

(…)-1.3B/resolve/main/tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

(…)dot_LLM-PLM-1.3B/resolve/main/vocab.json:   0%|          | 0.00/870k [00:00<?, ?B/s]

(…)dot_LLM-PLM-1.3B/resolve/main/merges.txt:   0%|          | 0.00/529k [00:00<?, ?B/s]

(…)LLM-PLM-1.3B/resolve/main/tokenizer.json:   0%|          | 0.00/2.25M [00:00<?, ?B/s]

(…)-PLM-1.3B/resolve/main/added_tokens.json:   0%|          | 0.00/95.0 [00:00<?, ?B/s]

(…).3B/resolve/main/special_tokens_map.json:   0%|          | 0.00/232 [00:00<?, ?B/s]

(…)ot_LLM-PLM-1.3B/resolve/main/config.json:   0%|          | 0.00/668 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.76G [00:00<?, ?B/s]

(…)1.3B/resolve/main/generation_config.json:   0%|          | 0.00/144 [00:00<?, ?B/s]

In [4]:
def generate(prefix, max_new_tokens=16, **kwargs):
  input_ids = tokenizer.encode(prefix, return_tensors="pt").to(device)
  outputs = model.generate(input_ids, max_new_tokens=max_new_tokens, **kwargs)
  print(tokenizer.decode(outputs[0]))

In [5]:
# greedy
generate(
    "오늘도 무척이나 심심한 하루였다. 뭐 재밌는 일이 없을까 고민하던 찰나,\n",
    )

# beam search (beam size=4)
generate(
    "오늘도 무척이나 심심한 하루였다. 뭐 재밌는 일이 없을까 고민하던 찰나,\n",
    num_beams=4
    )

오늘도 무척이나 심심한 하루였다. 뭐 재밌는 일이 없을까 고민하던 찰나,
"야, 너 오늘 뭐해?"
"응, 뭐해?"

오늘도 무척이나 심심한 하루였다. 뭐 재밌는 일이 없을까 고민하던 찰나,
"저기, 저기, 저기, 저기, 저기,


In [19]:
prompt = """수능날 긴장 안하는 꿀팁!
고삼 여러분 수능이 얼마 남지 않았습니다. 그동안 노력한 결실을 맺기 위해서 """

generate(
  prompt,
  do_sample=True
)

generate(
  prompt,
  do_sample=True,
  temperature=0.1
)

generate(
  prompt,
  do_sample=True,
  temperature=5.0
)

수능날 긴장 안하는 꿀팁!
고삼 여러분 수능이 얼마 남지 않았습니다. 그동안 노력한 결실을 맺기 위해서 ᅮ,_ᅮ 여러분들 모두 그동안 준비 잘하셨나요? 수능이 코
수능날 긴장 안하는 꿀팁!
고삼 여러분 수능이 얼마 남지 않았습니다. 그동안 노력한 결실을 맺기 위해서 ~~ 수능 당일 긴장하지 않고 시험을 잘 볼 수 있는 꿀팁을
수능날 긴장 안하는 꿀팁!
고삼 여러분 수능이 얼마 남지 않았습니다. 그동안 노력한 결실을 맺기 위해서 �량의 오늘하루 힘... 새 문서 인쇄하기...
2페이지|


In [7]:
prompt = """수능날 긴장 안하는 꿀팁!
고삼 여러분 수능이 얼마 남지 않았습니다. 그동안 노력한 결실을 맺기 위해서 """

generate(
  prompt,
  do_sample=True,
  top_k=50,
  top_p=0.7
)


수능날 긴장 안하는 꿀팁!
고삼 여러분 수능이 얼마 남지 않았습니다. 그동안 노력한 결실을 맺기 위해서 ~~~ 수능 당일에도 긴장하지 말고, 편안한 마음으로 시험을 치르시기 바랍니다


# In-context Learning

In [21]:
generate("""문장을 읽고 긍정/부정 중 하나를 선택하세요
이 영화는 엄복동 급의 명작입니다=""", 4)

문장을 읽고 긍정/부정 중 하나를 선택하세요
이 영화는 엄복동 급의 명작입니다=)
이 영화


In [22]:
generate("""문장을 읽고 긍정/부정 중 하나를 선택하세요
아 이 영화 진짜 개노잼이다=부정
올해 최고의 명작=긍정
이 영화는 엄복동 급의 명작입니다=""", 2)

문장을 읽고 긍정/부정 중 하나를 선택하세요
아 이 영화 진짜 개노잼이다=부정
올해 최고의 명작=긍정
이 영화는 엄복동 급의 명작입니다=긍정


In [24]:
generate("""문장을 존대말로 바꾸세요.
안녕 뭐하니=안녕하세요, 뭐하고 지내세요?
잘 잤냐=잘 주무셨어요?
안졸려?=졸리지 않으세요?
밥 먹었냐=""", 8, do_sample=True)

문장을 존대말로 바꾸세요.
안녕 뭐하니=안녕하세요, 뭐하고 지내세요?
잘 잤냐=잘 주무셨어요?
안졸려?=졸리지 않으세요?
밥 먹었냐=밥을 먹었니?
졸


In [25]:
generate("""문장을 읽고 긍정/부정 중 하나로 분류하세요
왕노잼=부정
왕꿀잼=긍정
핵노잼=""", 2)

문장을 읽고 긍정/부정 중 하나로 분류하세요
왕노잼=부정
왕꿀잼=긍정
핵노잼=부정


In [28]:
generate("""제시된 단어를 보고 창의적인 식당 리뷰를 만들어보세요

벌레=음식에 벌레나왔어요 ㅡㅡ 진짜 위생관리가 어떻게된건지 ;; 절대 여기서 주문하지 마세요.
반찬=배달시켰는데 반찬 몇개가 빠졌어요 ㅠㅠ
별점 5점=음식이 진짜 맛있어요
배달=""", 64, do_sample=True)

제시된 단어를 보고 창의적인 식당 리뷰를 만들어보세요

벌레=음식에 벌레나왔어요 ㅡㅡ 진짜 위생관리가 어떻게된건지 ;; 절대 여기서 주문하지 마세요.
반찬=배달시켰는데 반찬 몇개가 빠졌어요 ㅠㅠ
별점 5점=음식이 진짜 맛있어요
배달=배달의 민족이 뭔지 모르지만 일단 별로입니다
서비스=배달비도 안주고 배달안됨. 서비스로 주고받는게 있는거지.
양=양이 적어요
밥:짜서 많이 못먹음, 짜서 많이 못드십니다

4. 배달 음식점에서 가장


In [29]:
generate("""손흥민에 대한 뉴스 제목을 작성하세요
뉴스 제목: 손흥민, EPL 득점왕 달성할 수 있을까? 전문가들의 예상은?
뉴스 제목: 손흥민 풀타임, 이강인 강인함…‘가나전’ 승리 가나요
뉴스 제목: 후배들을 기 살려준 손흥민 "쫄지마, 너희도 잘하는 선수야"
뉴스 제목: """, max_new_tokens=32, do_sample=True)

손흥민에 대한 뉴스 제목을 작성하세요
뉴스 제목: 손흥민, EPL 득점왕 달성할 수 있을까? 전문가들의 예상은?
뉴스 제목: 손흥민 풀타임, 이강인 강인함…‘가나전’ 승리 가나요
뉴스 제목: 후배들을 기 살려준 손흥민 "쫄지마, 너희도 잘하는 선수야"
뉴스 제목: 伊언론, 손흥민에게 극찬 "세계 최고의 테크니션"
기자<|endoftext|>


In [30]:
generate(
    """이제부터 제가 재미있는 소설을 작성해보겠습니다.
내용: 요즘 우리 집안 분위기가 심상치 않다. """,
    max_new_tokens=512,
    do_sample=True,
    repetition_penalty=1.5
)

이제부터 제가 재미있는 소설을 작성해보겠습니다.
내용: 요즘 우리 집안 분위기가 심상치 않다. ᅮᅲ 특히 아저씨가 너무 안달난다!!! 왜냐구? 아빠가 제왕절개 수술 받으러 갔다 왔더니, 배에 자국이 남아서 계속 신경이 쓰인대서 말이다!!!!!! 그래서 나는 어젯밤 내내 인터넷에서 어떤 처녀애랑 섹스를 해도 된다는 그 미스터리 영화의 리뷰를 검색해서 엄마와 이 얘기를 했다.-_-;; 내가 생각해도 정말 이상한 애교(?)였다(그러곤 곧장 잠들어버렸다). 그러자 집에 있던 나머지 3명의 동생들도 다같이 침대에 누워서 저번 편과 똑 같은 방식으로 얘기하기 시작했다ᄏᄏᄏ 근데 이번에도 역시 내 말은 아무 의미도 없고 들으라는 얘기도 아니었다- _-) 하지만 나로 말할 것 같으면 솔직히 말하면 '아빠한테 성감형성수술 당하고 싶다'는 생각을 한 적이 있긴 있다! ᄒᄒᄒ 그리고 어느 날 밤새도록 꿈나라 여행을 하는 동안 갑자기 기발한 생각이 스쳐 지나갔다~! 그렇다면 이건 뭔가 대단하잖느냐???? (그날 이후 난 거의 매일 새벽마다 울리는 전화벨 소리에 깬다!) 지금 생각해 보면 그건 바로... *[주] 마시마야 로자리오.. 예쁘다~* 라고 했던 것이다!!!!! 뭐 이런 발칙하지만 우스운 녀석이 있는지??? ᄏᄏᄏ 하여튼 그렇게 해서 나도 드디어 완전체가 되어 버렸다는 게 문제지만............. 물론 여전히 지퍼백 안에 숨어있음....... ᅲ_^.<|endoftext|>


In [31]:
generate("""노래 제목과 장르를 보고, 사람들이 좋아할 만한 노래 가사를 써주세요.

노래 제목: 그대라는 시
장르: 발라드
가사:
언제부터인지 그대를 보면
운명이라고 느꼈던 걸까
밤하늘의 별이 빛난 것처럼
오랫동안 내 곁에 있어요
그대라는 시가 난 떠오를 때마다
외워두고 싶어 그댈 기억할 수 있게
슬픈 밤이 오면 내가 그대를 지켜줄게
내 마음 들려오나요 잊지 말아요

노래 제목: 밤편지
장르: 발라드/어쿠스틱
가사:
이 밤 그날의 반딧불을 당신의
""", max_new_tokens=128, do_sample=True, repetition_penalty=1.2)

노래 제목과 장르를 보고, 사람들이 좋아할 만한 노래 가사를 써주세요.

노래 제목: 그대라는 시
장르: 발라드
가사:
언제부터인지 그대를 보면
운명이라고 느꼈던 걸까
밤하늘의 별이 빛난 것처럼
오랫동안 내 곁에 있어요
그대라는 시가 난 떠오를 때마다
외워두고 싶어 그댈 기억할 수 있게
슬픈 밤이 오면 내가 그대를 지켜줄게
내 마음 들려오나요 잊지 말아요

노래 제목: 밤편지
장르: 발라드/어쿠스틱
가사:
이 밤 그날의 반딧불을 당신의
눈으로 느낄 때 나를 보내줘요
나는 기다리는 사람이에요 언제나와도 함께 있고 싶은 그런 사람이니까 나보다 더 소중한건 여기 없을 거예요 내게만 비밀인데 언제까지나 지금처럼 간직한다면 안 되겠니 사랑해요 이런 순간 처음인것 처럼 다시 시작해봐 오늘만은 오직 너 하나뿐야<|endoftext|>


In [15]:
generate("""게시글 제목과 가격을 보고, 판매 문구를 작성해주세요

제목: 맥북 프로 17인치 팝니다
가격: 1300000원
내용: 맥북 프로 산지 1년도 안됬는데 안써서 팝니다 어저구 저쩌구

제목: 애플워치 미개봉 팔아요
가격: 300000원
내용: 애플워치 잘못샀어요 ㅠㅠ 싸게 팔아요

제목: 엄마가 안쓰는 화분 팝니다
가격: 50000원
내용: """, max_new_tokens=128, do_sample=True, repetition_penalty=1.2)

게시글 제목과 가격을 보고, 판매 문구를 작성해주세요

제목: 맥북 프로 17인치 팝니다
가격: 1300000원
내용: 맥북 프로 산지 1년도 안됬는데 안써서 팝니다 어저구 저쩌구

제목: 애플워치 미개봉 팔아요
가격: 300000원
내용: 애플워치 잘못샀어요 ㅠㅠ 싸게 팔아요

제목: 엄마가 안쓰는 화분 팝니다
가격: 50000원
내용: ᄃᄉᄋ 화분이요 ᄒ 어머니 사시길래 대신 받앗네요. 깨긋합니다~
자세한 것은 쪽지로 연락 바랍니다<|endoftext|>


In [16]:
generate("""광고 내용을 보고 적절한 광고 문구를 만들어주세요

내용: 제주 생귤 1kg당 만원
광고 문구: 제초제를 전혀 사용하지 않고 나무에서 직접 딴 맛있는 제주 생귤 1kg 단돈 만원입니다.
내용: 하리보 포도맛 젤리
광고 문구: 아이들의 미소를 만드는 '하리보'의 포도맛 젤리! 초등학생들의 입맛을 사로잡은 그 맛, 이제 부모님들에게도 선사합니다.
내용: 원룸 청소업체
광고 문구: """, max_new_tokens=128, do_sample=True, repetition_penalty=1.2)

광고 내용을 보고 적절한 광고 문구를 만들어주세요

내용: 제주 생귤 1kg당 만원
광고 문구: 제초제를 전혀 사용하지 않고 나무에서 직접 딴 맛있는 제주 생귤 1kg 단돈 만원입니다.
내용: 하리보 포도맛 젤리
광고 문구: 아이들의 미소를 만드는 '하리보'의 포도맛 젤리! 초등학생들의 입맛을 사로잡은 그 맛, 이제 부모님들에게도 선사합니다.
내용: 원룸 청소업체
광고 문구: 月 10만원 (주 2회 출장) 한 달에 딱 한번만 맡겨주시면 편안하게 쉬세여~*^*(수납장 정리까지 해줌!!)*^* 집안일 고민말고 저희한테 시크릿콜 하세요^오^^_<=<|endoftext|>
